# Imports

In [1]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime

# Providing the Data

In [ ]:
train_dataset = torchvision.datasets.FashionMNIST(
    root='.',
    train=True,
    transform=transforms.ToTensor(),
    download=True
)

test_dataset = torchvision.datasets.FashionMNIST(
    root='.',
    train=False,
    transform=transforms.ToTensor(),
    download=True
)

In [10]:
batch_size = 128


train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False)

In [11]:
# number of classes
K = len(set(train_dataset.targets.numpy()))

# Model

In [12]:
class CNN(nn.Module):
  def __init__(self, K):
    super(CNN, self).__init__()
    self.conv_layers = nn.Sequential(
        nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3, stride=2),
        nn.ReLU(),
        nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=2),
        nn.ReLU(),
        nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, stride=2),
        nn.ReLU(),
    )
    # http://deeplearning.net/software/theano/tutorial/conv_arithmetic.html
    # https://pytorch.org/docs/stable/nn.html
    self.dense_layers = nn.Sequential(
        nn.Dropout(0.2),
        nn.Linear(128 * 2 * 2, 512),
        nn.ReLU(),
        nn.Dropout(0.2),
        nn.Linear(512, K)
    )

  def forward(self, x):
    out = self.conv_layers(x)
    out = out.view(out.size(0), -1)
    out = self.dense_layers(out)
    return out


In [13]:
model = CNN(K)

# Device, Loss, and Optimizer

In [14]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
model.to(device)

cuda:0


CNN(
  (conv_layers): Sequential(
    (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(2, 2))
    (1): ReLU()
    (2): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2))
    (3): ReLU()
    (4): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2))
    (5): ReLU()
  )
  (dense_layers): Sequential(
    (0): Dropout(p=0.2, inplace=False)
    (1): Linear(in_features=512, out_features=512, bias=True)
    (2): ReLU()
    (3): Dropout(p=0.2, inplace=False)
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)

In [15]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())

# Training - show duration

In [16]:
n_epochs = 10

train_losses = np.zeros(n_epochs)
test_losses = np.zeros(n_epochs)

for it in range(n_epochs):
  model.train()
  t0 = datetime.now()
  train_loss, test_loss = [], []
  for inputs, targets in train_loader:
    inputs, targets = inputs.to(device), targets.to(device)
    optimizer.zero_grad()
    outputs = model(inputs)
    loss = criterion(outputs, targets)
    loss.backward()
    optimizer.step()
    train_loss.append(loss.item())

  for inputs, targets in test_loader:
    inputs, targets = inputs.to(device), targets.to(device)
    outputs = model(inputs)
    loss = criterion(outputs, targets)
    test_loss.append(loss.item())

  train_losses[it] = np.mean(train_loss)
  test_losses[it] = np.mean(test_loss)
  dt = datetime.now() - t0
  print(f'Epoch {it + 1}/{n_epochs}, Train loss:{train_losses[it]:.4}, Test loss:{test_losses[it]:.4}, Duration:{dt}')

Epoch 1/10, Train loss:0.6593, Test loss:0.5091, Duration:0:00:07.639548
Epoch 2/10, Train loss:0.435, Test loss:0.4257, Duration:0:00:07.034246
Epoch 3/10, Train loss:0.3748, Test loss:0.382, Duration:0:00:07.111258
Epoch 4/10, Train loss:0.3386, Test loss:0.3575, Duration:0:00:07.140966
Epoch 5/10, Train loss:0.3107, Test loss:0.3401, Duration:0:00:07.123640
Epoch 6/10, Train loss:0.2912, Test loss:0.3246, Duration:0:00:07.504725
Epoch 7/10, Train loss:0.2673, Test loss:0.3301, Duration:0:00:07.426078
Epoch 8/10, Train loss:0.2532, Test loss:0.3074, Duration:0:00:07.663170
Epoch 9/10, Train loss:0.2373, Test loss:0.3259, Duration:0:00:08.197332
Epoch 10/10, Train loss:0.2243, Test loss:0.3056, Duration:0:00:07.055562
